In [3]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely import affinity
import requests
import zipfile
import io

# Load world countries
world_url = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json"
world = gpd.read_file(world_url)
countries = world[world['name'].isin(['India', 'Jordan'])].copy()

# Load US states (via census TIGER shapefile)
us_states_url = "https://www2.census.gov/geo/tiger/GENZ2022/shp/cb_2022_us_state_500k.zip"
r = requests.get(us_states_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("us_states_data")
us_states = gpd.read_file("us_states_data/cb_2022_us_state_500k.shp")

# Filter for California and Florida
states = us_states[us_states['NAME'].isin(['California', 'Florida'])].copy()
states = states.to_crs(epsg=4326)  # Match world CRS

# Standardize country/state names
states['name'] = states['NAME']
sel = pd.concat([countries[['name', 'geometry']], states[['name', 'geometry']]], ignore_index=True)

# Reproject to equal-area for area-based scaling
ea = sel.to_crs(epsg=6933)

# Compute area of India as reference
ref_area = ea.loc[ea['name'] == 'India', 'geometry'].area.values[0]
factors = {}
for idx, row in ea.iterrows():
    area = row.geometry.area
    factors[row['name']] = (ref_area / area) ** 0.5

# Scale about centroid
scaled = []
for idx, row in ea.iterrows():
    geom = row.geometry
    factor = factors[row['name']]
    cx, cy = geom.centroid.x, geom.centroid.y
    scaled_geom = affinity.scale(geom, xfact=factor, yfact=factor, origin=(cx, cy))
    scaled.append(scaled_geom)

ea['geometry'] = scaled

# Back to lat/lon
rescaled = ea.to_crs(epsg=4326)

# Plot
fig, ax = plt.subplots(figsize=(10, 7), dpi=300)
rescaled.plot(ax=ax, edgecolor='black', color='orange')

for name in rescaled['name']:
    print(f"Scaled: {name}, factor: {factors[name]:.2f}")

ax.set_axis_off()
plt.tight_layout()
plt.show()


SSLError: HTTPSConnectionPool(host='www2.census.gov', port=443): Max retries exceeded with url: /geo/tiger/GENZ2022/shp/cb_2022_us_state_500k.zip (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))